In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json

# Setup Chrome driver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

try:
    driver.implicitly_wait(10)
    url = "https://en.wikipedia.org/wiki/List_of_IOC_country_codes#Current_NOCs"
    driver.get(url)
    
    # Wait until the table is loaded
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[2]'))
    )
    
    # Get table element
    table = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[2]')
    
    # Initialize a dictionary to store the data
    country_data = {}

    # Iterate over each row in the table to extract data
    rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header row
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) > 1:  # Ensure the row has the required columns
            code = cols[0].text
            committee = cols[1].text
            country_data[code] = committee
    
    # Convert dictionary to JSON and save it to a file
    with open('Code2Country.json', 'w') as json_file:
        json.dump(country_data, json_file, indent=4)
        
finally:
    driver.quit()  # Ensure the driver is closed properly
